[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1a9Dqe9TNSwVXbw1zqUfOk0M8bwODYcBh)

如有需要，请取消下行注释，以安装 [geoca](https://pypi.org/project/geoca/)、[tqdm](https://pypi.org/project/tqdm/) 和 [pymannkendall](https://pypi.org/project/pymannkendall/)。

In [ ]:
# !pip install geoca tqdm pymannkendall

导入模块：**rasterconvert**、**tqdm** 和 **pymannkendall**。

使用 **rasterconvert** 转换栅格数据格式，使用 **tqdm** 在运行程序时创建进度条，使用 **pymannkendall** 执行 MK 趋势检验。

In [ ]:
from geoca import rasterconvert
from tqdm import tqdm
import pymannkendall as mk

**rasterconvert.multiRaster_listDict(folder_path, index_list)**：

- 从文件夹中读取多个 tif 格式栅格数据，并根据特定的索引值将其存储到字典中。

示例数据为某地区 1961-2022 年的降水量栅格数据，通过 multiRaster_listDict 函数读取每年的数据，并以列表形式将栅格输入数据字典。

In [ ]:
index_list = list(range(1961, 2022 + 1))
data_dict = rasterconvert.multiRaster_listDict("../Pre_Year", index_list)

In [ ]:
# 在结果字典中逐行输出 2011 年数据列表
# 2011 年数据列表示例：[[None, 1, 2],[4, 5, 3],[7, 6, None]]
for row in data_dict[2011]:
    print(row)

**rasterconvert.reorganize_multiRaster_listDict(data_dict, index_list)**：

- 以 multiRaster_listDict 函数为基础，对输入的字典数据进行重构，以便于后续处理。

In [ ]:
data_index_list, data_value_dict = rasterconvert.reorganize_multiRaster_listDict(data_dict, index_list)

In [ ]:
# 逐行输出 data_index_list 列表
# 该列表为二维索引列表，行列数与原始栅格数据相同，元素值为水平坐标和垂直坐标的元组
# data_index_list 列表示例： [[(1, 1), (1, 2), (1, 3)],[(2, 1), (2, 2), (2, 3)],[(3, 1), (3, 2), (3, 3)]]
for row in data_index_list:
    print(row)

In [ ]:
# 输出 data_value_dict 字典中第一个值中不包含 None 的键值对
# 该字典的键为坐标元组（data_index_list 列表元素值），值是与坐标相对应的所有栅格图像元素值的列表（按 index_list 列表顺序排列）
# 输出示例：(2, 3): [3, 2, 6, 7, 9]
for index, data_list in data_value_dict.items():
    if None in data_list:
        continue
    print(f"{index}: {data_list}")
    break

转换完成后，可根据这些数据对相应的多个栅格像元值进行叠加分析。例如，**pymannkendall** 模块可用于对时间序列气象栅格数据进行 MK 趋势检验，以获得每个图像元素的 slope 值：

In [ ]:
# 根据 data_dict 字典中任意列表的行列数创建空列表模板
key1 = list(data_dict.keys())[0]
list1 = data_dict[key1]
rows, cols = len(list1), len(list1[0])
slope_list = [[None for _ in range(cols)] for _ in range(rows)]

# 遍历 data_value_dict 字典，对字典中的列表执行 MK 趋势检验，并将 slope 依次添加到列表模板中的对应位置
for index, data_list in tqdm(data_value_dict.items()):
    if None in data_list:
        continue
    i, j = index
    mkTest = mk.original_test(data_list)
    slope_list[i][j] = mkTest.slope

**rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, nodata_value)**：

- 根据输入栅格创建栅格模板，并用新数据替换像素值来处理栅格数据。

In [ ]:
rasterconvert.process_raster_data("../Pre_Year/Pre_2011.tif", "../MKData/mkSlope.tif", slope_list, None)